<a href="https://colab.research.google.com/github/ShehzadAli005/ShehzadAli005/blob/main/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate sacrebleu
!pip install  sentencepiece sacremoses -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 26.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... 

In [2]:
from datasets import load_dataset
books = load_dataset("opus_books", "de-en")
books = books["train"].train_test_split(test_size=0.2)
books["train"][0]

Generating train split:   0%|          | 0/51467 [00:00<?, ? examples/s]

{'id': '12081',
 'translation': {'de': 'Die alte Frau nahm sie und sah sie sich lange an, und ein Lächeln des Glückes sonnte ihr Gesicht. Als sie wegging, hörte man sie vor sich hinmurmeln: »Ich werde sie dem Herrn Pfarrer bei uns zu Hause geben, damit er mir dermaleinst eine Messe liest.«',
  'en': 'Then, when she had her medal, she looked at it, and a smile of beatitude spread over her face; and as she walked away they could hear her muttering "I\'ll give it to our cure up home, to say some masses for me!"'}}

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, DataCollatorForSeq2Seq, AdamWeightDecay, TFAutoModelForSeq2SeqLM
# from transformers import MarianMTModel, MarianTokenizer
checkpoint = "google/t5-v1_1-small"
tokenizer = T5Tokenizer.from_pretrained(checkpoint)
# tokenizer = MarianTokenizer.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
source_lang = "de" # Deutsch (German)
target_lang = "en" # English
prefix = "translate German to English: "
def preprocess_encoder(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

tokenized_books = books.map(preprocess_encoder, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

Map:   0%|          | 0/41173 [00:00<?, ? examples/s]

Map:   0%|          | 0/10294 [00:00<?, ? examples/s]

In [5]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at google/t5-v1_1-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [6]:
# Run inference using pretrained weights
inputs = tokenizer(
    "translate German to English: Wie geht es dir", return_tensors="tf"
).input_ids  # Batch size 1
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
# studies have shown that owning a dog is good for you

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:834: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


..hl as.


In [7]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_books["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_books["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator
)

In [8]:
model.compile(optimizer=optimizer)  # No loss argument!
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

Epoch 1/3
2573/2573 [==============================] - 841s 315ms/step - loss: 6.3036 - val_loss: 4.0590
Epoch 2/3
2573/2573 [==============================] - 817s 318ms/step - loss: 4.9535 - val_loss: 3.7067
Epoch 3/3
2573/2573 [==============================] - 815s 317ms/step - loss: 4.5459 - val_loss: 3.5122


In [11]:
source_texts = ["Wie geht es dir",
                "Warum bist du ein Zurückgebliebener?"
                ""]
input_ids = tokenizer(source_texts, return_tensors="tf", padding=True, truncation=True)["input_ids"]

translated_ids = model.generate(
    input_ids=input_ids,
    max_length=100,   # Maximum length of the generated translation
    num_beams=4,      # Number of beams for beam search
    early_stopping=True,
    no_repeat_ngram_size=2,  # Avoid repeating n-grams in the generated translation
)

translated_texts = tokenizer.batch_decode(translated_ids, skip_special_tokens=True)
print(translated_texts)

# from transformers import pipeline

# translator = pipeline("translation", model=model, tokenizer=tokenizer)
# translator(text)

['Then he asked me.', 'Then what is it?']


In [12]:
# Optional

import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result